In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake-and-real-news-dataset/True.csv
/kaggle/input/fake-and-real-news-dataset/Fake.csv


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings("ignore")
import os
from tqdm.notebook import tqdm
from ast import literal_eval
import pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [4]:
fake_df = pd.read_csv(r'/kaggle/input/fake-and-real-news-dataset/Fake.csv')
true_df = pd.read_csv(r'/kaggle/input/fake-and-real-news-dataset/True.csv')

In [5]:
fake_df['class'] = 0 
true_df['class'] = 1

df = pd.concat([fake_df, true_df], ignore_index=True)

In [6]:
df.drop_duplicates(inplace=True)

In [7]:
X = df[["text"]]
y = df[["class"]]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.8, stratify=y, random_state=41
)

# Transformer 

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments
from datasets import Dataset
import torch

# 1. Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# 2. Convert your dataset into Hugging Face Dataset format
# Assume X_train, y_train, X_test, y_test are pandas Series
train_ds = Dataset.from_dict({"text": list(X_train['text']), "label": y_train['class'].tolist()})
test_ds = Dataset.from_dict({"text": list(X_test['text']), "label": y_test['class'].tolist()})

# 3. Tokenization
def tokenize_function(batch):
    return tokenizer(batch["text"], truncation=True, padding=True, max_length=256)

tokenized_train = train_ds.map(tokenize_function, batched=True)
tokenized_test = test_ds.map(tokenize_function, batched=True)

# 4. Load pre-trained DistilBERT for binary classification
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2
)

# 5. Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 6. Training arguments
training_args = TrainingArguments(
    output_dir="./results",
        learning_rate=2e-5,                # usually better to keep small
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    eval_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_dir="./logs",
)

# 7. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# 8. Train (Fine-Tuning)
trainer.train()

# 9. Save fine-tuned model
trainer.save_model("model")

Map:   0%|          | 0/35751 [00:00<?, ? examples/s]

Map:   0%|          | 0/8938 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss
1,0.014100,0.000202
2,0.000600,0.000040
3,0.000300,0.000029


In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Define metric computation
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

# Re-create trainer with compute_metrics
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Evaluate on test set
results = trainer.evaluate()
print(results)

{'eval_loss': 2.906800727942027e-05, 'eval_accuracy': 1.0, 'eval_precision': 1.0, 'eval_recall': 1.0, 'eval_f1': 1.0, 'eval_runtime': 43.5392, 'eval_samples_per_second': 205.286, 'eval_steps_per_second': 6.431}
